In [12]:
import numpy as np
import pandas as pd

from skmultiflow.meta import OzaBaggingADWINClassifier
from skmultiflow.lazy import KNNClassifier
from sklearn.preprocessing import StandardScaler

### Uso do KNN com um método de detecção de Drift

In [13]:
clf = OzaBaggingADWINClassifier(base_estimator=KNNClassifier(metric='euclidean', n_neighbors=3), n_estimators=2)
sample_count = 0
corrects = 0

### Carregamento dos dados de 2011

In [14]:
enade_treino = pd.read_csv("../concept_drift/tabela_final_2011_treinamento.csv")

numero_caracteristicas = enade_treino.shape[1] - 1
X_enade_treino = enade_treino.iloc[:, 0:numero_caracteristicas]
y_enade_treino = enade_treino.iloc[:, -1]

normalizador_treinamento = StandardScaler()

normalizador_treinamento.fit(X_enade_treino)
treinamento_normalizado = normalizador_treinamento.transform(X_enade_treino)

clf = clf.partial_fit(treinamento_normalizado, y_enade_treino, classes=[1,2,3,4,5])

### Carregamento dos dados de 2014, 2017 e 2021

In [16]:
enade_teste = pd.read_csv("../concept_drift/tabela_final_14-17-21_treinamento.csv")

numero_caracteristicas = enade_teste.shape[1] - 1
X_enade_teste = enade_teste.iloc[:, 0:numero_caracteristicas]
y_enade_teste = enade_teste.iloc[:, -1]

normalizador_teste = StandardScaler()

normalizador_teste.fit(X_enade_teste)
teste_normalizado = normalizador_teste.transform(X_enade_teste)

<class 'pandas.core.series.Series'>


### Execução do código quickstart (falha)

In [65]:
for index, linha in enumerate(teste_normalizado):
    pred = clf.predict(linha)
    clf = clf.partial_fit(pd.DataFrame([linha], columns=np.arange(168)), [y_enade_teste.values[index]])
    if pred is not None:
        if y_enade_teste.values[index] == pred[0]:
            corrects += 1
    sample_count += 1

ValueError: Inconsistent number of features in X: 1, previously observed 168.

In [ ]:
print(str(sample_count) + ' samples analyzed.')
print('OzaBaggingADWINClassifier performance: ' + str(corrects / sample_count))